In [28]:
# Importing necessary libraries and modules

# Standard library imports
import asyncio  # For asynchronous operations
import time  # For time-based operations and timestamps
import os  # For environment management and file operations
import warnings  # To filter out warnings
import nest_asyncio # To avoid runtime errors related to asynchronus opreations

# Importing to fetch environment variables
from dotenv import load_dotenv

# Third-party library imports
import matplotlib.pyplot as plt  # For plotting and visualization
import numpy as np  # For numerical operations and array manipulations
import pandas as pd  # For data manipulation and analysis
import pandas_ta as ta  # For technical analysis indicators and tools
import joblib  # For model serialization and deserialization
import ccxt  # For cryptocurrency trading APIs and market data retrieval

# Machine Learning libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier  # For ML model training
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler  # For feature scaling
from sklearn.metrics import accuracy_score, classification_report  # For model evaluation

# Google Cloud imports
from google.cloud import bigquery, storage  # For interacting with Google BigQuery and Storage
from google.oauth2 import service_account  # For Google Cloud authentication

# Empyreal SDK imports for strategy development and backtesting
from emp_orderly import Strategy, EmpOrderly  # For strategy implementation and management
from emp_orderly_types import PerpetualAssetType, Interval  # For defining asset types and intervals

warnings.filterwarnings('ignore')

In [29]:
load_dotenv()

True

In [36]:
# Class for implementing a trading strategy
class AlphaWaveTrader(Strategy):
    def init(self):
        """
        Initialize the strategy by loading the pre-trained model and scaler from Google Cloud Storage.
        Also, pre-calculate all technical indicators used for generating buy/sell signals.
        """
        # Load GCP credentials and project information from environment variables
        credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
        project_id = os.getenv("GCP_PROJECT_ID")

        # Ensure that both environment variables are set
        if not project_id or not credentials_path:
            raise ValueError("GCP_PROJECT_ID and GOOGLE_APPLICATION_CREDENTIALS must be set in the .env file.")

        # Initialize Google Cloud service account credentials using the specified JSON file
        credentials = service_account.Credentials.from_service_account_file(credentials_path)

        # Initialize the Google Cloud Storage client
        storage_client = storage.Client(credentials=credentials, project=project_id)

        # Access the bucket containing the model and scaler files
        bucket = storage_client.bucket(f'{project_id}-crypto_trading_bucket')

        # Download the pre-trained model file from the bucket
        model_blob = bucket.blob('trading_model.pkl')
        model_blob.download_to_filename('trading_model.pkl')
        self.model = joblib.load('trading_model.pkl')  # Load the trained model

        # Download the scaler file from the bucket
        scaler_blob = bucket.blob('scaler.pkl')
        scaler_blob.download_to_filename('scaler.pkl')
        self.scaler = joblib.load('scaler.pkl')  # Load the scaler

        # Prepare a DataFrame with technical indicators for the strategy
        df = pd.DataFrame({
            'close': self.data.close,
            'high': self.data.high,
            'low': self.data.low
        })

        # Add technical indicators to the DataFrame
        df['SMA_10'] = ta.sma(df['close'], length=10)
        df['SMA_20'] = ta.sma(df['close'], length=20)
        df['RSI'] = ta.rsi(df['close'], length=14)

        # Add MACD and Bollinger Bands indicators
        macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
        df['MACD'] = macd['MACD_12_26_9']
        df['MACD_signal'] = macd['MACDs_12_26_9']
        bbands = ta.bbands(df['close'], length=20)
        df['BB_upper'] = bbands['BBU_20_2.0']
        df['BB_middle'] = bbands['BBM_20_2.0']
        df['BB_lower'] = bbands['BBL_20_2.0']

        # Add additional indicators like ATR, Momentum, and ROC
        df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)
        df['MOM'] = ta.mom(df['close'], length=10)
        df['ROC'] = ta.roc(df['close'], length=10)

        # Drop rows with NaN values that result from indicator calculations
        df.dropna(inplace=True)

        # Store the precomputed indicator data for use in the strategy
        self.indicator_data = df.reset_index(drop=True)

    def next(self):
        """
        Generate buy/sell signals based on the machine learning model's predictions using the latest data.
        """
        # Get the index of the last data point in the current context
        idx = len(self.data.close) - 1

        # Ensure there is enough data for making predictions
        if idx < len(self.indicator_data):
            # Extract the row of technical indicator values for the current index
            row = self.indicator_data.iloc[idx]

            # Define the feature set used for model predictions
            features = [
                row['SMA_10'], row['SMA_20'], row['RSI'], row['MACD'], row['MACD_signal'],
                row['BB_upper'], row['BB_middle'], row['BB_lower'], row['ATR'], row['MOM'], row['ROC']
            ]

            # Scale the features using the pre-loaded scaler
            scaled_features = self.scaler.transform([features])

            # Generate a prediction using the pre-trained model
            prediction = self.model.predict(scaled_features)[0]

            # Implement buy/sell logic based on the prediction
            if prediction == 1:
                # Buy signal: Close short positions and open a long position
                if self.position.is_short:
                    self.position.close()
                if not self.position.is_long:
                    self.buy(size=1)
            else:
                # Sell signal: Close long positions and open a short position
                if self.position.is_long:
                    self.position.close()
                if not self.position.is_short:
                    self.sell(size=1)

In [48]:
# Main function for backtesting the custom strategy
async def main():
    """
    Main function to set up the EmpOrderly SDK, load the custom trading strategy,
    and perform backtesting with historical data.
    """
    # Create a new instance of EmpOrderly with specified parameters
    sdk = EmpOrderly(
        cash=1000,  # Initial cash amount for the backtesting
        commission=0.0001,  # Commission rate per trade
        exclusive_orders=True  # Ensure only one position is active at a time
    )

    # Set the custom trend-following strategy to the SDK and load historical data
    sdk.set_strategy(AlphaWaveTrader)

    # Load historical data for backtesting with a lookback period and specified asset
    await sdk.load_data(
        lookback=5,
        interval=Interval.thirty_minute,
        asset=PerpetualAssetType.BNB,
    )

    # Perform backtesting with the loaded data and strategy
    stats = sdk.backtest()
    print(stats)

    # sdk.plot(show_price_data=False)
    # plt.show()

# Entry point of the script
if __name__ == "__main__":
    nest_asyncio.apply()

    # Run the main asynchronous function until it completes
    asyncio.get_event_loop().run_until_complete(main())

Start                     2024-10-02 04:00:00
End                       2024-10-07 03:30:00
Duration                      4 days 23:30:00
Exposure Time [%]                   99.166667
Equity Final [$]                  1022.547061
Equity Peak [$]                   1025.207061
Return [%]                           2.254706
Buy & Hold Return [%]                3.439857
Return (Ann.) [%]                  553.439283
Volatility (Ann.) [%]               42.436936
Sharpe Ratio                        13.041453
Sortino Ratio                             inf
Calmar Ratio                       308.449742
Max. Drawdown [%]                   -1.794261
Avg. Drawdown [%]                   -0.428369
Max. Drawdown Duration        2 days 04:30:00
Avg. Drawdown Duration        0 days 08:40:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       6.348229
Worst Trade [%]                     -1.819476
Avg. Trade [%]                    